# C++ Cling Examples for STM32L053Nucleo

![Image](img/head.png)

Load [REMCU](https://remotemcu.com) shared libray

In [ ]:
.L libremcu

Add path with header files

In [ ]:
.I remcu_include/

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [ ]:
#include "remcu.h"
#include "stm32l0xx_hal.h"
#include "stm32l0xx_ll_bus.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.com/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.com/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)

Before that, plug stm32 board with debugger to PC

run openocd utility in command line:
```
openocd -f board/stm32l0discovery.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative
bool connection = false;

In [ ]:
connection = remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//connection = remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

In [ ]:
if(connection == false){
    fprintf(stderr, "1. Check connection to debug server\n");
    fprintf(stderr, "2. Check debug server running\n");
    fprintf(stderr, "3. Check the debugged MCU is correct\n");
    exit(1);
}

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)):

In [ ]:
remcu_resetRemoteUnit(__HALT)

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (startup_stm32f4xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f4xx.c file

In [ ]:
SystemInit();

This sample code shows how to use STM32L0xx GPIO HAL API to toggle PA5 Pin (connected to LED2, on STM32L0xx_Nucleo board) in an infinite loop. To proceed, 3 steps are required:

STM32L0xx HAL library initialization:

   - Configure the Flash prefetch, Flash preread and Buffer caches
   - In the default implementation , SysTick timer is configured as main source 
     of time base. It is used to generate interrupts at 1ms basis.
   - Low Level Initialization

In [ ]:
HAL_Init();

### GPIO Example
![Image](img/gpio.png)
>The code below have been taken from **IO_Toggle** ([main.c](STM32CubeL0_example_projects/NUCLEO-L053R8/Examples/GPIO/GPIO_IOToggle/Src/main.c)) example of [STM32L0 HAL and Low Layer drivers](https://www.st.com/resource/en/user_manual/dm00113898-description-of-stm32l0-hal-and-low-layer-drivers-stmicroelectronics.pdf)

Clock enable and configure pins:

In [ ]:
  RCC_ClkInitTypeDef RCC_ClkInitStruct;
  RCC_OscInitTypeDef RCC_OscInitStruct;
  
  /* Enable MSI Oscillator */
  RCC_OscInitStruct.OscillatorType = RCC_OSCILLATORTYPE_MSI;
  RCC_OscInitStruct.MSIState = RCC_MSI_ON;
  RCC_OscInitStruct.MSIClockRange = RCC_MSIRANGE_5;
  RCC_OscInitStruct.MSICalibrationValue=0x00;
  RCC_OscInitStruct.PLL.PLLState = RCC_PLL_NONE;
  HAL_RCC_OscConfig(&RCC_OscInitStruct);
  
  
  /* Select MSI as system clock source and configure the HCLK, PCLK1 and PCLK2 
     clocks dividers */
  RCC_ClkInitStruct.ClockType = (RCC_CLOCKTYPE_SYSCLK | RCC_CLOCKTYPE_HCLK | RCC_CLOCKTYPE_PCLK1 | RCC_CLOCKTYPE_PCLK2);
  RCC_ClkInitStruct.SYSCLKSource = RCC_SYSCLKSOURCE_MSI;
  RCC_ClkInitStruct.AHBCLKDivider = RCC_SYSCLK_DIV1;
  RCC_ClkInitStruct.APB1CLKDivider = RCC_HCLK_DIV1;  
  RCC_ClkInitStruct.APB2CLKDivider = RCC_HCLK_DIV1;  
  HAL_RCC_ClockConfig(&RCC_ClkInitStruct, FLASH_LATENCY_0);

In [ ]:
  /* -1- Enable GPIOA Clock (to be able to program the configuration registers) */
  //__HAL_RCC_GPIOA_CLK_ENABLE(); - don't use __HAL_RCC_GPIOA_CLK_ENABLE define
LL_IOP_GRP1_EnableClock(LL_IOP_GRP1_PERIPH_GPIOA);

Configure PA.5 IO in output push-pull mode to drive external LEDs 

In [ ]:
 GPIO_InitTypeDef  GPIO_InitStruct;  
/* -2- */
  GPIO_InitStruct.Pin = (GPIO_PIN_5);
  GPIO_InitStruct.Mode = GPIO_MODE_OUTPUT_PP;
  GPIO_InitStruct.Pull = GPIO_PULLUP;
  GPIO_InitStruct.Speed = GPIO_SPEED_FREQ_HIGH  ;
  
  HAL_GPIO_Init(GPIOA, &GPIO_InitStruct); 

Flash leds on:

In [ ]:
HAL_GPIO_WritePin(GPIOA, GPIO_PIN_5, GPIO_PIN_SET)

<details>
  <summary><b>Result(click to show)</b></summary>

![Image](img/led_on.png)
</details>

Flash leds off:

In [ ]:
HAL_GPIO_WritePin(GPIOA, GPIO_PIN_5, GPIO_PIN_RESET)

If it didn’t happen, you can use the utility functions to detect errors...

### Utility Functions

The ***remcu_getErrorCount***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function returns the number of errors that occurred after connection:

In [ ]:
remcu_getErrorCount()

If no error occurred, result ***0***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)

The ***remcu_isConnected***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function indicates status of the current debug server connection:

In [ ]:
remcu_isConnected()

Result: connected ***true*** , disconnected ***false***

Some of debug adapters can have hardware problems. The ***remcu_debuggerTest*** (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function enables to detect the such adapters. The function returns a null pointer if the test is successful:

In [ ]:
remcu_debuggerTest()

If no error occurred, result ***nullptr***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)

>If bugs are not discovered and the chip don't responds to driver function call or the result of work was not what you expected, see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)


### ADC example
![adc_head](img/adc_head.png )
>The code below is based on **ADC_RegularConversion_Polling** ([main.c](STM32CubeL0_example_projects/NUCLEO-L053R8/Examples/ADC/ADC_RegularConversion_Polling/Src/main.c)) example of [STM32L0 HAL and Low Layer drivers](https://www.st.com/resource/en/user_manual/dm00113898-description-of-stm32l0-hal-and-low-layer-drivers-stmicroelectronics.pdf)

clock config for ADC:

In [ ]:
#include "stm32l0xx_ll_rcc.h"
#include "stm32l0xx_ll_system.h"

/* Set APB1 & APB2 prescaler*/
LL_RCC_SetAPB1Prescaler(LL_RCC_APB1_DIV_1);
LL_RCC_SetAPB2Prescaler(LL_RCC_APB2_DIV_1);

enable ADC system clock:

In [ ]:
 /* Enable ADC clock (core clock) */
  LL_APB2_GRP1_EnableClock(LL_APB2_GRP1_PERIPH_ADC1);

GPIOA Periph clock enable:

In [ ]:
  /* Enable GPIO Clock */
  LL_IOP_GRP1_EnableClock(LL_IOP_GRP1_PERIPH_GPIOA);

Configure PA.00 (ADC Channel0) as analog input:

In [ ]:
GPIO_InitStruct.Pin = GPIO_PIN_0;
GPIO_InitStruct.Mode = GPIO_MODE_ANALOG;
GPIO_InitStruct.Pull = GPIO_NOPULL;
HAL_GPIO_Init(GPIOA, &GPIO_InitStruct);

In [ ]:
bool errorflag = false;

/* ADC handle declaration */
ADC_HandleTypeDef             AdcHandle;

/* ADC channel configuration structure declaration */
ADC_ChannelConfTypeDef        adcConfig;

uint32_t uwADCxConvertedValue = 0;

#####  Initialize ADC peripheral 

   *  Instance                  = ADC1.
   *  OversamplingMode          = Disabled
   *  ClockPrescaler            = PCLK clock with no division.
   *  LowPowerAutoPowerOff      = Disabled (For this exemple continuous mode is enabled with software start)
   *  LowPowerFrequencyMode     = Enabled (To be enabled only if ADC clock is lower than 2.8MHz) 
   *  LowPowerAutoWait          = Disabled (New conversion starts only when the previous conversion is completed)       
   *  Resolution                = 12 bit (increased to 16 bit with oversampler)
   *  SamplingTime              = 7.5 cycles od ADC clock.
   *  ScanConvMode              = Forward
   *  DataAlign                 = Right
   *  ContinuousConvMode        = Enabled
   *  DiscontinuousConvMode     = Disabled
   *  ExternalTrigConvEdge      = None (Software start)
   *  EOCSelection              = End Of Conversion event
   *  DMAContinuousRequests     = DISABLE

In [ ]:
  AdcHandle.Instance = ADC1;
  
  AdcHandle.Init.OversamplingMode      = DISABLE;
  
  AdcHandle.Init.ClockPrescaler        = ADC_CLOCK_SYNC_PCLK_DIV1;
  AdcHandle.Init.LowPowerAutoPowerOff  = DISABLE;
  AdcHandle.Init.LowPowerFrequencyMode = ENABLE;
  AdcHandle.Init.LowPowerAutoWait      = DISABLE;
    
  AdcHandle.Init.Resolution            = ADC_RESOLUTION_12B;
  AdcHandle.Init.SamplingTime          = ADC_SAMPLETIME_7CYCLES_5;
  AdcHandle.Init.ScanConvMode          = ADC_SCAN_DIRECTION_FORWARD;
  AdcHandle.Init.DataAlign             = ADC_DATAALIGN_RIGHT;
  AdcHandle.Init.ContinuousConvMode    = ENABLE;
  AdcHandle.Init.DiscontinuousConvMode = DISABLE;
  AdcHandle.Init.ExternalTrigConvEdge  = ADC_EXTERNALTRIGCONVEDGE_NONE;
  AdcHandle.Init.EOCSelection          = ADC_EOC_SINGLE_CONV;
  AdcHandle.Init.DMAContinuousRequests = DISABLE;
 
  /* Initialize ADC peripheral according to the passed parameters */
  if (HAL_ADC_Init(&AdcHandle) != HAL_OK)
  {
    fprintf(stderr,"Error\n");
    errorflag = true;
  }
  
  
  /* ### - 2 - Start calibration ############################################ */
  if (HAL_ADCEx_Calibration_Start(&AdcHandle, ADC_SINGLE_ENDED) != HAL_OK)
  {
    fprintf(stderr,"Error\n");
    errorflag = true;
  }

##### Channel configuration 
  * Select Channel 0 to be converted

In [ ]:
  adcConfig.Channel = ADC_CHANNEL_0;    
  if (HAL_ADC_ConfigChannel(&AdcHandle, &adcConfig) != HAL_OK)
  {
    fprintf(stderr,"Error\n");
    errorflag = true;
  }
  
 /*##- 4- Start the conversion process #######################################*/  
  if(HAL_ADC_Start(&AdcHandle) != HAL_OK)
  {
    /* Start Conversation Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
  }

##### Wait for the end of conversion 
  Before starting a new conversion, you need to check the current state of
         the peripheral; if it’s busy you need to wait for the end of current
         conversion before starting a new one.
         For simplicity reasons, this example is just waiting till the end of the
         conversion, but application may perform other tasks while conversion
         operation is ongoing.

In [ ]:
uint16_t ADC_value(){
    HAL_ADC_PollForConversion(&AdcHandle, 10);

    /* Check if the continous conversion of regular channel is finished */
    while ((HAL_ADC_GetState(&AdcHandle) & HAL_ADC_STATE_REG_EOC) != HAL_ADC_STATE_REG_EOC);
    /*##-6- Get the converted value of regular channel  ########################*/
    return HAL_ADC_GetValue(&AdcHandle);
}

ADC_value()

Prepare canvas, which shows ADC data. [Template](https://github.com/QuantStack/xplot/blob/3399c4b96dffb1fec9d9c44de74d3d223bb04789/notebooks/xplot.ipynb)
> First install [xplot](https://github.com/QuantStack/xplot) or make sure you have it
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [ ]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

const size_t ADC_BUFF_SIZE = 100;
xpl::figure fig;
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
std::vector<uint16_t> adc_data(ADC_BUFF_SIZE);
std::vector<int> x_line(ADC_BUFF_SIZE);
int i = 0;
for( auto & x : x_line){x = i++;}
line.x = x_line;
line.y = adc_data;
fig.add_mark(line);
    
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);
fig

Plug a potentiometer into the PA0 pin and run the code below. Rotate the shaft and you will get the voltage versus time chart:

In [ ]:
#include <unistd.h>

for(size_t i = 0; i < 100; i++){
    adc_data[i % ADC_BUFF_SIZE] = ADC_value();
    //plot ADC data
    line.y = adc_data;
    usleep(200000);
}

<details>
  <summary>Result<b>(click to show)</b></summary>

![Image](img/adc_cpp.png)
</details>

### Extra Utility Function

The ***ReMCU_getVersion***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function returns a pointer to a char array containing information about the REMCU lib versions

In [ ]:
remcu_getVersion()

The ***ReMCU_disconnect***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function disconnects from the debug server. It is finalizing work with REMCU Library.

In [ ]:
remcu_disconnect()